<a href="https://colab.research.google.com/github/cdelaoutre/rerA-waze/blob/master/pythonTests/scrapTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports ###

In [17]:
!pip install tweepy
import tweepy
import json
import csv
from datetime import date
from datetime import datetime
import time

*Twitter API Connection Infos manually loaded*

In [18]:
# Load Twitter API secrets from an external JSON file
with open("twitterInfos.json", "r") as read_file:
    connInfos = json.load(read_file)

api_key = connInfos['twitter_key']
api_secret_key = connInfos['twitter_secret']
access_token = connInfos['twitter_accessToken']
access_token_secret = connInfos['twitter_accessTokenSecret']
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)